In [11]:
import subprocess
import os
import re

def run_lammps_simulation(input_file, output_file, seed):
    # Modify the input file to include the random seed
    with open(input_file, 'r') as file:
        data = file.readlines()
    
    # Find the line where the random seed is set and modify it
    for i, line in enumerate(data):
        if 'velocity     all create' in line:
            data[i] = f"velocity     all create 300.0 {seed} mom yes rot yes dist gaussian\n"
            break
    
    # Write the modified input file
    with open(input_file, 'w') as file:
        file.writelines(data)
    
    # Run the LAMMPS simulation
    subprocess.run(['lmp_serial', '-in', input_file])

    # After running, extract the thermal conductivity value from the output file
    with open(output_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            # Look for the line that contains 'average conductivity'
            if 'average conductivity' in line:
                # Extract the numeric value using a regex pattern
                match = re.search(r'average conductivity: ([0-9.e+-]+)', line)
                if match:
                    k_value = float(match.group(1))  # Extracted value
                    return k_value
    
    return None

def run_simulations(input_file, output_file, num_runs):
    results = []
    for i in range(num_runs):
        seed = 102486 + i  # You can use any formula for the seed, or randomize
        print(f"Running simulation {i+1} with seed {seed}")
        k_value = run_lammps_simulation(input_file, output_file, seed)
        if k_value is not None:
            results.append(k_value)
        else:
            print(f"Simulation {i+1} failed to extract conductivity.")
    
    return results

# Define the paths to files
input_file = 'LJsim_diffatoms.txt'
output_file = 'log.lammps'  # output filename

# Run the simulations
results = run_simulations(input_file, output_file, 10)  # Run 10 simulations

# Print the results
print("Thermal conductivity results:")
for i, k in enumerate(results, 1):
    print(f"Run {i}: {k} W/mK")

Running simulation 1 with seed 102486
Running simulation 2 with seed 102487
Running simulation 3 with seed 102488
Running simulation 4 with seed 102489
Running simulation 5 with seed 102490
Running simulation 6 with seed 102491
Running simulation 7 with seed 102492
Running simulation 8 with seed 102493
Running simulation 9 with seed 102494
Running simulation 10 with seed 102495
Thermal conductivity results:
Run 1: 0.291769297141165 W/mK
Run 2: 0.328044290602053 W/mK
Run 3: 0.271378479247555 W/mK
Run 4: 0.246476772220032 W/mK
Run 5: 0.236773774884863 W/mK
Run 6: 0.270039294238975 W/mK
Run 7: 0.240671374351345 W/mK
Run 8: 0.296936705540613 W/mK
Run 9: 0.257436036439933 W/mK
Run 10: 0.326600912930482 W/mK


In [13]:
import csv

# Save the results to a CSV file
with open('thermal_conductivity_results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Run", "Thermal Conductivity (W/mK)"])  # Write headers
    for i, k in enumerate(results, 1):
        writer.writerow([i, k])  # Write each result as a new row